#### using `pymongo`

In [2]:
import json
from bson.json_util import dumps
from pymongo import MongoClient

# initialize a client object
client = MongoClient("localhost", 27017)
client.list_database_names()

# select db
db = client["mongoTest"] # or client.mongoTest
db.list_collection_names()

# select collection
movies = db["movies"]
# run query >> returns iterable cursor
result = movies.find({"release_year" : {"$gt" : 2000}})

#### print `cursor` documents

In [3]:
# print documents from cursor
for x in result:
    print(dumps(x, indent=4))

{
    "_id": {
        "$oid": "691c1609585f06670943a6c5"
    },
    "name": "The Dark Knight",
    "leads": [
        "Christian Bale",
        "Heath Ledger",
        "Aaron Eckhart"
    ],
    "release_year": 2008
}
{
    "_id": {
        "$oid": "691c1609585f06670943a6c8"
    },
    "name": "The Lord of the Rings: The Return of the King",
    "leads": [
        "Noel Appleby",
        "Ali Astin",
        "Sean Astin"
    ],
    "release_year": 2003
}


#### define `projection`

In [7]:
# projections
query = {"release_year" : {"$lt" : 2000}}
projection = {"_id": 0, "leads": 1}
for x in movies.find(query, projection):
    print(dumps(x, indent=4))

{
    "leads": [
        "Tim Robbins",
        "Morgan Freeman",
        "Bob Gunton"
    ]
}
{
    "leads": [
        "Marlon Brando",
        "Al Pacino",
        "James Caan"
    ]
}
{
    "leads": [
        "Martin Balsam",
        "John Fiedler",
        "Lee J. Cobb"
    ]
}


#### get query plan

In [ ]:
movies.find({"release_year" : {"$gt" : 2000}}).explain() # COLLSCAN

{'explainVersion': '1',
 'queryPlanner': {'namespace': 'mongoTest.movies',
  'parsedQuery': {'release_year': {'$gt': 2000}},
  'indexFilterSet': False,
  'queryHash': 'B3289643',
  'planCacheShapeHash': 'B3289643',
  'planCacheKey': 'D474F1DA',
  'optimizationTimeMillis': 0,
  'maxIndexedOrSolutionsReached': False,
  'maxIndexedAndSolutionsReached': False,
  'maxScansToExplodeReached': False,
  'prunedSimilarIndexes': False,
  'winningPlan': {'isCached': False,
   'stage': 'COLLSCAN',
   'filter': {'release_year': {'$gt': 2000}},
   'direction': 'forward'},
  'rejectedPlans': []},
 'executionStats': {'executionSuccess': True,
  'nReturned': 2,
  'executionTimeMillis': 0,
  'totalKeysExamined': 0,
  'totalDocsExamined': 5,
  'executionStages': {'isCached': False,
   'stage': 'COLLSCAN',
   'filter': {'release_year': {'$gt': 2000}},
   'nReturned': 2,
   'executionTimeMillisEstimate': 0,
   'works': 6,
   'advanced': 2,
   'needTime': 3,
   'needYield': 0,
   'saveState': 0,
   'restoreS

#### create `index`

In [ ]:
movies.create_index("release_year")
movies.find({"release_year" : {"$gt" : 2000}}).explain() # changes to IXSCAN

{'explainVersion': '1',
 'queryPlanner': {'namespace': 'mongoTest.movies',
  'parsedQuery': {'release_year': {'$gt': 2000}},
  'indexFilterSet': False,
  'queryHash': 'B3289643',
  'planCacheShapeHash': 'B3289643',
  'planCacheKey': '6DB88D91',
  'optimizationTimeMillis': 0,
  'maxIndexedOrSolutionsReached': False,
  'maxIndexedAndSolutionsReached': False,
  'maxScansToExplodeReached': False,
  'prunedSimilarIndexes': False,
  'winningPlan': {'isCached': False,
   'stage': 'FETCH',
   'inputStage': {'stage': 'IXSCAN',
    'keyPattern': {'release_year': 1},
    'indexName': 'release_year_1',
    'isMultiKey': False,
    'multiKeyPaths': {'release_year': []},
    'isUnique': False,
    'isSparse': False,
    'isPartial': False,
    'indexVersion': 2,
    'direction': 'forward',
    'indexBounds': {'release_year': ['(2000, inf]']}}},
  'rejectedPlans': []},
 'executionStats': {'executionSuccess': True,
  'nReturned': 2,
  'executionTimeMillis': 2,
  'totalKeysExamined': 2,
  'totalDocsExa